# Problema
$$ min \ c_1 ^T x + c_2 ^T y $$
$$ Subject. to: \ A_1 x + A_2 y \le b $$
$$ x \ge 0 $$
$$ y \ge 0 $$
$$ x \in Z ^n $$

# Main Problem
$$ min \ c_1 ^T x + \theta $$
$$ Subject. to: \ x \ge 0 $$
$$ x \in Z ^n $$
$$ \theta \ge \theta ^k - \lambda A_1 ( x - x ^k) $$

# Sub Problem
$$ \theta = min \  c_2 ^T y $$
$$ Subject. to: \  A_2 y \le b - A_1 x \ [\lambda]$$
$$ y \ge 0 $$

In [67]:
using JuMP, GLPK, Printf

In [68]:
c1=[1,4]
c2=[2,3]
b2=[-2;-3]
A1=[1 -3; -1 -3]
A2=[1 -2; -1 -1]
M=-1000

-1000

# Modeling de main problem

In [70]:
main = Model(GLPK.Optimizer)
@variable(main,0 ≤ x[1:2], Int)
@variable(main,-1000 ≤ θ)
@objective(main, Min, c1' * x + θ)
print(main)

Min x[1] + 4 x[2] + θ
Subject to
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 θ ≥ -1000.0
 x[1] integer
 x[2] integer


# Modeling subproblem

In [71]:
function sub(x)
    sub = Model(GLPK.Optimizer)
    @variable(sub, 0 ≤ y[1:2])
    @objective(sub, Min, c2' * y)
    @constraint(sub, A1 * x + A2 * y .≤ b2)
    optimize!(sub)
    o = objective_value(sub)
    y = value.(y)
    all_con = all_constraints(sub, AffExpr, MOI.LessThan{Float64})
    λ = dual.(all_con)
    return Dict('o' => o , 'y' => y , 'λ' => λ)
end    

sub (generic function with 1 method)

In [72]:
function print_iteracion(k, args...)
    f(x) = Printf.@sprintf("%12.4e",x)
    println(lpad(k,9), " ", join(f.(args), " "))
    return
end

print_iteracion (generic function with 1 method)

# Iteraciones
- $c_1 x + c_2 y$ Objetive original problem
- $c_1 x^k + \theta$ lower bound (k iterativa)
- $c_1 x^k + c_2 y^k$  Upper bound

$$ \theta \ge \theta ^k - \lambda A_1 ( x - x ^k) $$

In [73]:
println("        k     upperbound     lowerbound      gap")
for k in 1:10
    optimize!(main)
    lb = objective_value(main)
    xᵏ = value.(x)
    ub = c1' * xᵏ + c2' * sub(xᵏ)['y']
    gap = (ub-lb)/ub
    print_iteracion(k, lb, ub, gap)
    if gap < 0.0000001
        println("*********** felicitaciones optimalidad encontrada **********************")
        break
    end
    benderscut = @constraint(main, θ ≥ sub(xᵏ)['o'] - (sub(xᵏ)['λ'])' * A1 * (x .- xᵏ))
    @info "Añadiendo este corte $(benderscut)"
end

        k     upperbound     lowerbound      gap
        1  -1.0000e+03   7.6667e+00   1.3143e+02
        2  -4.9600e+02   1.2630e+03   1.3927e+00
        3  -1.0800e+02   8.8800e+02   1.1216e+00
        4   4.0000e+00   4.0000e+00   0.0000e+00
*********** felicitaciones optimalidad encontrada **********************


┌ Info: Añadiendo este corte 2 x[1] + 8 x[2] + θ ≥ 7.666666666666666
└ @ Main In[73]:14
┌ Info: Añadiendo este corte -1.5 x[1] + 4.5 x[2] + θ ≥ 3.0
└ @ Main In[73]:14
┌ Info: Añadiendo este corte θ ≥ 0.0
└ @ Main In[73]:14


In [74]:
print(main)

Min x[1] + 4 x[2] + θ
Subject to
 2 x[1] + 8 x[2] + θ ≥ 7.666666666666666
 -1.5 x[1] + 4.5 x[2] + θ ≥ 3.0
 θ ≥ 0.0
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 θ ≥ -1000.0
 x[1] integer
 x[2] integer


In [65]:
value.(x)

2-element Array{Float64,1}:
 0.0
 1.0